In [1]:
import pandas as pd
from annotate_papers import *

In [2]:
metadata_df = pd.read_excel("21699/dataframe21699.xlsx")
metadata_df.head()

,author,title,affiliation,email,abstract,year,journal,doi,street,city,zip,filenames
0,"Michael L. Brodie, Michael Stonebraker",DARWIN: On the Incremental Migration of Legacy...,Hochschule Albstadt-Sigmaringen,michael_l._brodie@hochschulealbsta.de,befragungen von mitarbeitern der verwaltung ha...,1997,ZA-Information / Zentralarchiv für Empirische ...,https://doi.org/10.1080/00036840701736115,Schoenebergerstrasse 79,Stralsund,18412,
1,Frank Manola,Integrating Object-Oriented Applications and M...,Ostbayerische Technische Hochschule Amberg-Weiden,frank_manola@ostbayerischetechnisc.de,Das International Social Survey Programme (ISS...,2009,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Albrechtstrasse 84,Nußdorf,83131,
2,Frank Manola,Object Data Language Facilities for Multimedia...,Hochschule für angewandte Wissenschaften Augsburg,frank_manola@hochschulef.de,Die Gesellschaft Sozialwissenschaftlicher Infr...,2001,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Anhalter Strasse 18,Hauptstuhl,66851,
3,"Frank Manola, Mark F. Hornick, Alejandro P. Bu...",Object Data Model Facilities for Multimedia Da...,Universität Augsburg,frank_manola@universitätaugsburg..de,"Der eigene Körper ist das uns Nächste, das die...",2004,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Lange Strasse 31,Ottobrunn,85501,
4,Frank Manola,Distributed Object Management Technology.,Hochschule der Bayerischen Wirtschaft (München),frank_manola@hochschuled.de,Die vorliegende Studie berichtet über die Durc...,1997,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Lützowplatz 25,Irsch,54451,


In [3]:
metadata_df.rename(columns={'year':'date', 'city': 'address'}, inplace=True)

In [4]:
metadata_df[metadata_df.filenames == "21699_1063.docx"]

,author,title,affiliation,email,abstract,date,journal,doi,street,address,zip,filenames
3363,"Dhrubajit Chowdhury, Hassan K. Khalil",Fast Consensus in Multi-Agent Systems With Sta...,SRH Fernhochschule Riedlingen,dhrubajit_chowdhury@srhfernhochschu.de,"Der eigene Körper ist das uns Nächste, das die...",1981,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Sömmeringstr. 54,Pforzheim Weststadt,75172,21699_1063.docx


# create a subclass for 2169

In [5]:
class Template_21699(Paper):
    def __init__(self, filename, metadata_dict):
        super().__init__(filename, metadata_dict)
        
    def bbox_from_text(self):
        # get the first page
        doc = fitz.open(self.filename)
        page = doc[0]
        
        if list(self.metadata.values())[0] == "":
            raise MetadataError("No metadata values specified. Run method get_metadata_items(metadata_df) before callsing this method.", "")
        
        ending_box = None
        
        for key, val in self.metadata.items():
            if key == "title":
                beginning_box = page.searchFor("no 01/")
                ending_box = page.searchFor(self.metadata["author"])   
                title_rects = []
                for word in page.getTextWords():
                    if str.lower(word[4]) in str.lower(val):
                        title_rects.append(fitz.Rect(word[:4]))

                self.rectangles[key] = [rect for rect in title_rects if rect[3] > beginning_box[0][3] and rect[3] < ending_box[0][3]]
            elif key == "abstract":
                beginning_box_abstract = page.searchFor("Keywords")
                abstract_rects = page.searchFor(val, hit_max=1000)
                if len(abstract_rects) == 0:
                    text = self.metadata[key].split("-")
                    for part in text:
                        results = page.searchFor(part, hit_max=1000)
                        abstract_rects += results
                #for word in page.getTextWords():
                #    if str.lower(word[4]) in str.lower(val):
                #        abstract_rects.append(fitz.Rect(word[:4]))
                
                self.rectangles[key] = [rect for rect in abstract_rects if rect[3] > beginning_box_abstract[0][3]]
            else:
                self.rectangles[key] = page.searchFor(val, hit_max=1000)
                if len(self.rectangles[key]) == 0:
                    text = self.metadata[key].split("-")
                    for part in text:
                        results = page.searchFor(part, hit_max=1000)
                        self.rectangles[key] += results
                        
            if len(self.rectangles[key]) > 1:
                foundRectList = [tuple(rect) for rect in self.rectangles[key]]
    
                #find min x, min y, max x, max y
                min_x = foundRectList[0][0]
                min_y = foundRectList[0][1]
                max_x = foundRectList[0][2]
                max_y = foundRectList[0][3]

                for rect in foundRectList:
                    if rect[0] < min_x:
                        min_x = rect[0]
                    if rect[1] < min_y:
                        min_y = rect[1]
                    if rect[2] > max_x:
                        max_x = rect[2]
                    if rect[3] > max_y:
                        max_y = rect[3]
            
                self.rectangles[key] = [fitz.Rect(min_x, min_y, max_x, max_y)]
                
        if ending_box != None and len(ending_box) > 1:
            ending_box = [rect for rect in ending_box if rect[3] > self.rectangles["date"][0][3] and rect[3] < self.rectangles["author"][0][3]]
            self.rectangles["title"] = [rect for rect in self.rectangles["title"] if rect[3] < ending_box[0][3]]
        if len(self.rectangles["title"]) > 1:
            foundRectList = [tuple(rect) for rect in self.rectangles["title"]]

            #find min x, min y, max x, max y
            min_x = foundRectList[0][0]
            min_y = foundRectList[0][1]
            max_x = foundRectList[0][2]
            max_y = foundRectList[0][3]

            for rect in foundRectList:
                if rect[0] < min_x:
                    min_x = rect[0]
                if rect[1] < min_y:
                    min_y = rect[1]
                if rect[2] > max_x:
                    max_x = rect[2]
                if rect[3] > max_y:
                    max_y = rect[3]

            self.rectangles["title"] = [fitz.Rect(min_x, min_y, max_x, max_y)]


In [6]:
p = Template_21699("21699/21699_912.pdf",{"title": "", "author":"", "abstract": "", "date": "01/", "affiliation": "", "email": ""})

In [7]:
p.resize_image(569,794)
p.save_image()

In [8]:
p.get_metadata_items(metadata_df)
p.metadata

{'title': 'Mesoscience-based virtual process engineering.',
 'author': 'Wei Ge, Li Guo , Xinhua Liu, Fanyong Meng , Ji Xu, Wenlai Huang, Jinghai Li',
 'abstract': "Ausgegangen wird von Lacans These, wonach in der 'phantasmatischen' Wahrnehmung das erkenntnissuchende Subjekt nicht die empirische Realität sucht, sondern sich selbst vermittels der Spiegelung in den Objekten und Subjekt-Objekt-Beziehungen der Ähnlichkeit hergestellt werden auf der Grundlage von Identifikation und Projektion. Dieses 'imaginäre' Denken flüchtet vor dem Fragmentarischen in die hierarchische Ordnung des Symbolischen und in die Unterscheidung bzw. Abgrenzung von 'Ich' und dem 'Anderen'. Die Namen und die von ihnen markierten Differenzen werden nach dieser repräsentativen 'Spiegellogik' vor die Erkenntnisobjekte gesetzt, die auf diese Weise zwar ihren hierarchischen Platz im Symbolischen erhalten, in ihrem Wesen aber somit nicht verstanden werden können. Im Fortschreiten dieses Prozesses entwickelt sich das 'Spi

In [9]:
p.bbox_from_text()
p.rectangles

[Rect(57.02399826049805, 268.0821228027344, 539.9449462890625, 281.3687744140625), Rect(57.02399826049805, 279.6020812988281, 497.77410888671875, 292.88873291015625), Rect(410.74346923828125, 279.6020812988281, 437.8546447753906, 292.88873291015625), Rect(501.5799865722656, 279.6020812988281, 528.810546875, 292.88873291015625), Rect(330.5257263183594, 325.5621032714844, 355.58514404296875, 338.8487548828125), Rect(177.9384002685547, 371.5220947265625, 205.16900634765625, 384.8087463378906), Rect(57.02399826049805, 291.0021057128906, 495.2643737792969, 304.28875732421875), Rect(57.02399826049805, 302.5220642089844, 515.6199951171875, 315.8087158203125), Rect(57.02399826049805, 314.0420837402344, 531.3692626953125, 327.3287353515625), Rect(57.02399826049805, 325.5621032714844, 520.927490234375, 338.8487548828125), Rect(57.02399826049805, 337.08209228515625, 507.9433288574219, 350.3687438964844), Rect(57.02399826049805, 348.6020812988281, 290.7154235839844, 361.88873291015625), Rect(294.3

{'title': [Rect(57.02399826049805, 110.51718139648438, 379.8902587890625, 129.8502655029297)],
 'author': [Rect(57.02399826049805, 135.10397338867188, 492.239990234375, 151.68797302246094)],
 'abstract': [Rect(57.02399826049805, 268.0821228027344, 539.9449462890625, 453.8287353515625)],
 'date': [Rect(284.76202392578125, 52.980003356933594, 328.14202880859375, 69.46800231933594)],
 'affiliation': [Rect(57.02399826049805, 155.99295043945312, 207.59481811523438, 169.67799377441406)],
 'email': [Rect(227.08999633789062, 155.99295043945312, 349.98638916015625, 169.67799377441406)]}

In [10]:
p.get_annotations()
p.save_annotations()

In [32]:
doc = fitz.open("21699/21699_912.pdf")
page = doc[0]

In [38]:
page.searchFor("Ausgegangen wird von Lacans These, wonach in der 'phantasmatischen' Wahrnehmung das erkenntnissuchende Subjekt nicht die empirische Realität sucht, sondern sich selbst vermittels der Spiegelung in den Objekten und Subjekt-Objekt-Beziehungen der Ähnlichkeit hergestellt werden auf der Grundlage von Identifikation und Projektion. Dieses 'imaginäre' Denken flüchtet vor dem Fragmentarischen in die hierarchische Ordnung des Symbolischen und in die Unterscheidung bzw. Abgrenzung von 'Ich' und dem 'Anderen'. Die Namen und die von ihnen markierten Differenzen werden nach dieser repräsentativen 'Spiegellogik' vor die Erkenntnisobjekte gesetzt, die auf diese Weise zwar ihren hierarchischen Platz im Symbolischen erhalten, in ihrem Wesen aber somit nicht verstanden werden können. Im Fortschreiten dieses Prozesses entwickelt sich das 'Spiegel-Ich' zum 'sozialen Ich'. Dieser Weg der Identitätsfindung wird als Grundmuster männlichen Denkens in der abendländischen Kultur beschrieben. Es wird gezeigt, wie dieser instrumentelle Zugriff auf die Objekte erfolgt unter negativer Besetzung alles Weiblichen, das als Repräsentanz des Chaotischen begriffen wird. Rückgreifend auf Levi-Strauss wird die totemistische Logik des 'Wilden Denkens' und ihres semantischen Zugangs zur Komplexität der Welt durch das Spannen von Fäden zwischen den Dingen dem klassischen 'Beherrschungs-Denken' entgegengesetzt. Hier wird Welt als kohärente Ordnung begriffen, in der alle Elemente in einem sinnhaften und sinnvollen wurzelartig verbundenen System (Rhizom) auf komplexe Weise in Zusammenhang stehen. Es geht in der Krise der modernen Humanwissenschaft darum, nicht die alten Denk-Bilder zu bewachen, sondern sich Bilder kreativ und verstehend zuzuspielen. (ICB)Ausgegangen wird von Lacans These, wonach in der 'phantasmatischen' Wahrnehmung das erkenntnissuchende Subjekt nicht die empirische Realität sucht, sondern sich selbst vermittels der Spiegelung in den Objekten und Subjekt")# -Objekt-Beziehungen der Ähnlichkeit hergestellt werden auf der Grundlage von Identifikation und Projektion. Dieses 'imaginäre' Denken flüchtet vor dem Fragmentarischen in die hierarchische Ordnung des Symbolischen und in die Unterscheidung bzw. Abgrenzung von 'Ich' und dem 'Anderen'. Die Namen und die von ihnen markierten Differenzen werden nach dieser repräsentativen 'Spiegellogik' vor die Erkenntnisobjekte gesetzt, die auf diese Weise zwar ihren hierarchischen Platz im Symbolischen erhalten, in ihrem Wesen aber somit nicht verstanden werden können. Im Fortschreiten dieses Prozesses entwickelt sich das 'Spiegel-Ich' zum 'sozialen Ich'. Dieser Weg der Identitätsfindung wird als Grundmuster männlichen Denkens in der abendländischen Kultur beschrieben. Es wird gezeigt, wie dieser instrumentelle Zugriff auf die Objekte erfolgt unter negativer Besetzung alles Weiblichen, das als Repräsentanz des Chaotischen begriffen wird. Rückgreifend auf Levi-Strauss wird die totemistische Logik des 'Wilden Denkens' und ihres semantischen Zugangs zur Komplexität der Welt durch das Spannen von Fäden zwischen den Dingen dem klassischen 'Beherrschungs-Denken' entgegengesetzt. Hier wird Welt als kohärente Ordnung begriffen, in der alle Elemente in einem sinnhaften und sinnvollen wurzelartig verbundenen System (Rhizom) auf komplexe Weise in Zusammenhang stehen. Es geht in der Krise der modernen Humanwissenschaft darum, nicht die alten Denk-Bilder zu bewachen, sondern sich Bilder kreativ und verstehend zuzuspielen. (ICB)")

[]

In [ ]:
for word in page.getTextWords():
    if str.lower(word[4]) in str.lower('CFD-DEM-PBM coupled model development and validation of a 3D top-spray fluidized bed wet granulation process.'):
        print(word)

# annotate all pdfs for template 2169

In [6]:
from glob import glob

In [7]:
pdfs = glob("21699/*.pdf")
pdfs

['21699/21699_80.pdf',
 '21699/21699_951.pdf',
 '21699/21699_25.pdf',
 '21699/21699_55.pdf',
 '21699/21699_875.pdf',
 '21699/21699_611.pdf',
 '21699/21699_1262.pdf',
 '21699/21699_914.pdf',
 '21699/21699_82.pdf',
 '21699/21699_1533.pdf',
 '21699/21699_450.pdf',
 '21699/21699_920.pdf',
 '21699/21699_1167.pdf',
 '21699/21699_923.pdf',
 '21699/21699_840.pdf',
 '21699/21699_999.pdf',
 '21699/21699_1541.pdf',
 '21699/21699_1470.pdf',
 '21699/21699_973.pdf',
 '21699/21699_358.pdf',
 '21699/21699_864.pdf',
 '21699/21699_233.pdf',
 '21699/21699_941.pdf',
 '21699/21699_1333.pdf',
 '21699/21699_1214.pdf',
 '21699/21699_252.pdf',
 '21699/21699_546.pdf',
 '21699/21699_909.pdf',
 '21699/21699_486.pdf',
 '21699/21699_1374.pdf',
 '21699/21699_968.pdf',
 '21699/21699_1259.pdf',
 '21699/21699_627.pdf',
 '21699/21699_195.pdf',
 '21699/21699_196.pdf',
 '21699/21699_1410.pdf',
 '21699/21699_694.pdf',
 '21699/21699_1119.pdf',
 '21699/21699_1038.pdf',
 '21699/21699_511.pdf',
 '21699/21699_691.pdf',
 '21699/

In [8]:
errors = []
for pdf in pdfs:
    
    print(pdf)
    
    p = Template_21699(pdf, {"title": "", "author":"", "abstract": "", "date": "01/", "affiliation": "", "email": ""})
    try:
        p.resize_image(569,794)
        p.save_image("21699_annotations/")
        p.get_metadata_items(metadata_df)
        p.bbox_from_text()
        #print(p.rectangles)
        p.get_annotations()
        p.save_annotations("21699_annotations/")
    except:
        errors.append(pdf)
        pass


21699/21699_80.pdf
21699/21699_951.pdf
21699/21699_25.pdf
21699/21699_55.pdf
21699/21699_875.pdf
21699/21699_611.pdf
21699/21699_1262.pdf
21699/21699_914.pdf
21699/21699_82.pdf
21699/21699_1533.pdf
21699/21699_450.pdf
21699/21699_920.pdf
21699/21699_1167.pdf
21699/21699_923.pdf
21699/21699_840.pdf
21699/21699_999.pdf
21699/21699_1541.pdf
21699/21699_1470.pdf
21699/21699_973.pdf
21699/21699_358.pdf
21699/21699_864.pdf
21699/21699_233.pdf
21699/21699_941.pdf
21699/21699_1333.pdf
21699/21699_1214.pdf
21699/21699_252.pdf
21699/21699_546.pdf
21699/21699_909.pdf
21699/21699_486.pdf
21699/21699_1374.pdf
21699/21699_968.pdf
21699/21699_1259.pdf
21699/21699_627.pdf
21699/21699_195.pdf
21699/21699_196.pdf
21699/21699_1410.pdf
21699/21699_694.pdf
21699/21699_1119.pdf
21699/21699_1038.pdf
21699/21699_511.pdf
21699/21699_691.pdf
21699/21699_811.pdf
21699/21699_621.pdf
21699/21699_434.pdf
21699/21699_299.pdf
21699/21699_112.pdf
21699/21699_159.pdf
21699/21699_96.pdf
21699/21699_170.pdf
21699/21699_8

21699/21699_326.pdf
21699/21699_63.pdf
21699/21699_1124.pdf
21699/21699_994.pdf
21699/21699_570.pdf
21699/21699_1232.pdf
21699/21699_1015.pdf
21699/21699_816.pdf
21699/21699_292.pdf
21699/21699_1589.pdf
21699/21699_827.pdf
21699/21699_848.pdf
21699/21699_954.pdf
21699/21699_820.pdf
21699/21699_320.pdf
21699/21699_590.pdf
21699/21699_924.pdf
21699/21699_1122.pdf
21699/21699_1089.pdf
21699/21699_1293.pdf
21699/21699_759.pdf
21699/21699_1468.pdf
21699/21699_187.pdf
21699/21699_1228.pdf
21699/21699_1011.pdf
21699/21699_1107.pdf
21699/21699_1472.pdf
21699/21699_259.pdf
21699/21699_1555.pdf
21699/21699_832.pdf
21699/21699_485.pdf
21699/21699_1204.pdf
21699/21699_1477.pdf
21699/21699_1319.pdf
21699/21699_293.pdf
21699/21699_1357.pdf
21699/21699_1261.pdf
21699/21699_1133.pdf
21699/21699_1265.pdf
21699/21699_577.pdf
21699/21699_817.pdf
21699/21699_1552.pdf
21699/21699_892.pdf
21699/21699_117.pdf
21699/21699_90.pdf
21699/21699_567.pdf
21699/21699_565.pdf
21699/21699_1297.pdf
21699/21699_1045.pdf

21699/21699_16.pdf
21699/21699_684.pdf
21699/21699_945.pdf
21699/21699_376.pdf
21699/21699_370.pdf
21699/21699_1573.pdf
21699/21699_708.pdf
21699/21699_472.pdf
21699/21699_793.pdf
21699/21699_1090.pdf
21699/21699_599.pdf
21699/21699_729.pdf
21699/21699_843.pdf
21699/21699_685.pdf
21699/21699_792.pdf
21699/21699_1544.pdf
21699/21699_352.pdf
21699/21699_724.pdf
21699/21699_460.pdf
21699/21699_933.pdf
21699/21699_534.pdf
21699/21699_1569.pdf
21699/21699_1510.pdf
21699/21699_575.pdf
21699/21699_1139.pdf
21699/21699_524.pdf
21699/21699_1069.pdf
21699/21699_83.pdf
21699/21699_1138.pdf
21699/21699_78.pdf
21699/21699_1505.pdf
21699/21699_815.pdf
21699/21699_41.pdf
21699/21699_1286.pdf
21699/21699_331.pdf
21699/21699_207.pdf
21699/21699_1458.pdf
21699/21699_785.pdf
21699/21699_307.pdf
21699/21699_1007.pdf
21699/21699_695.pdf
21699/21699_1184.pdf
21699/21699_649.pdf
21699/21699_1491.pdf
21699/21699_629.pdf
21699/21699_1288.pdf
21699/21699_322.pdf
21699/21699_551.pdf
21699/21699_949.pdf
21699/216

21699/21699_88.pdf
21699/21699_950.pdf
21699/21699_410.pdf
21699/21699_1279.pdf
21699/21699_445.pdf
21699/21699_121.pdf
21699/21699_137.pdf
21699/21699_427.pdf
21699/21699_1452.pdf
21699/21699_1480.pdf
21699/21699_58.pdf
21699/21699_1157.pdf
21699/21699_478.pdf
21699/21699_939.pdf
21699/21699_549.pdf
21699/21699_1081.pdf
21699/21699_1159.pdf
21699/21699_1006.pdf
21699/21699_499.pdf
21699/21699_769.pdf
21699/21699_260.pdf
21699/21699_786.pdf
21699/21699_194.pdf
21699/21699_1453.pdf
21699/21699_188.pdf
21699/21699_1287.pdf
21699/21699_1475.pdf
21699/21699_654.pdf
21699/21699_95.pdf
21699/21699_1466.pdf
21699/21699_1336.pdf
21699/21699_1568.pdf
21699/21699_669.pdf
21699/21699_181.pdf
21699/21699_444.pdf
21699/21699_386.pdf
21699/21699_639.pdf
21699/21699_1378.pdf
21699/21699_1033.pdf
21699/21699_1582.pdf
21699/21699_783.pdf
21699/21699_1199.pdf
21699/21699_129.pdf
21699/21699_531.pdf
21699/21699_276.pdf
21699/21699_537.pdf
21699/21699_1459.pdf
21699/21699_217.pdf
21699/21699_175.pdf
21699

In [9]:
errors

['21699/21699_346.pdf',
 '21699/21699_682.pdf',
 '21699/21699_405.pdf',
 '21699/21699_519.pdf',
 '21699/21699_361.pdf',
 '21699/21699_729.pdf',
 '21699/21699_885.pdf',
 '21699/21699_9.pdf',
 '21699/21699_365.pdf',
 '21699/21699_778.pdf',
 '21699/21699_1558.pdf',
 '21699/21699_85.pdf',
 '21699/21699_593.pdf',
 '21699/21699_697.pdf',
 '21699/21699_226.pdf']

In [10]:
import os
for pdf in errors:
    print(pdf[6:-4] + ".jpeg")
    jpeg = "21699_annotations/" + pdf[6:-4] + ".jpeg"
    json = "21699_annotations/" + pdf[6:-4] + ".json"
    if os.path.exists(jpeg):
        os.remove(jpeg)
        print("removed jpeg: " + jpeg)
    if os.path.exists(json):
        os.remove(json)
        print("removed json: " + json)
    

21699_346.jpeg
removed jpeg: 21699_annotations/21699_346.jpeg
21699_682.jpeg
removed jpeg: 21699_annotations/21699_682.jpeg
21699_405.jpeg
removed jpeg: 21699_annotations/21699_405.jpeg
21699_519.jpeg
removed jpeg: 21699_annotations/21699_519.jpeg
21699_361.jpeg
removed jpeg: 21699_annotations/21699_361.jpeg
21699_729.jpeg
removed jpeg: 21699_annotations/21699_729.jpeg
21699_885.jpeg
removed jpeg: 21699_annotations/21699_885.jpeg
21699_9.jpeg
removed jpeg: 21699_annotations/21699_9.jpeg
21699_365.jpeg
removed jpeg: 21699_annotations/21699_365.jpeg
21699_778.jpeg
removed jpeg: 21699_annotations/21699_778.jpeg
21699_1558.jpeg
removed jpeg: 21699_annotations/21699_1558.jpeg
21699_85.jpeg
removed jpeg: 21699_annotations/21699_85.jpeg
21699_593.jpeg
removed jpeg: 21699_annotations/21699_593.jpeg
21699_697.jpeg
removed jpeg: 21699_annotations/21699_697.jpeg
21699_226.jpeg
removed jpeg: 21699_annotations/21699_226.jpeg


# running some tests

In [ ]:
p = Paper("21699/21699_1410.pdf", {"title": "", "author":"", "abstract": "", "date": "01/", "affiliation": "", "email": ""})

In [ ]:
p.resize_image(569,794)
p.save_image()

In [ ]:
p.get_metadata_items(metadata_df)
p.metadata

In [ ]:
p.bbox_from_text()
p.rectangles

In [ ]:
p.get_annotations()

In [ ]:
p.save_annotations()

In [ ]:
    p = Template_21699(pdf, {"title": "", "author":"", "abstract": "", "date": "01/", "affiliation": "", "email": ""})
    p.resize_image(569,794)
    p.save_image("21699_annotations/")
    p.get_metadata_items(metadata_df)
    p.bbox_from_text()
    #print(p.rectangles)
    p.get_annotations()
    p.save_annotations("21699_annotations/")